In [ ]:
# TASK 1
# 1. Import thư viện cần thiết
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 2. Định nghĩa tên file
filename = "50_Startups.csv"

# 3. Kiểm tra nếu file chưa tồn tại thì yêu cầu tải lên
if not os.path.exists(filename):
    from google.colab import files
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]  # Lấy tên file vừa tải lên

# 4. Đọc file CSV vào DataFrame
df = pd.read_csv(filename)

# 5. Loại bỏ các cột chứa dữ liệu không phải số
df_numeric = df.select_dtypes(include=[np.number])  # Chỉ giữ lại các cột số

# 6. Hiển thị 5 dòng đầu tiên để kiểm tra dữ liệu
print("\n📌 Dữ liệu số ban đầu:")
print(df_numeric.head())

# 7. Tính toán các thống kê cơ bản
summary = df_numeric.describe()
print("\n📌 Thống kê dữ liệu:\n", summary)

# 8. Vẽ scatter plot giữa từng feature và biến phụ thuộc y
y = df_numeric.iloc[:, -1]  # Cột cuối cùng là biến phụ thuộc (y)
features = df_numeric.columns[:-1]  # Các cột còn lại là feature

for feature in features:
    plt.scatter(df_numeric[feature], y)
    plt.xlabel(feature)
    plt.ylabel("Output (y)")
    plt.title(f'Scatter plot: {feature} vs Output')
    plt.show()

# 9. Vẽ histogram của từng feature
for feature in df_numeric.columns:
    mean = df_numeric[feature].mean()
    var = df_numeric[feature].var()
    plt.hist(df_numeric[feature], bins=20, alpha=0.7, color='blue')
    plt.title(f'Histogram of {feature} (mean={mean:.1f}, var={var:.1f})')
    plt.xlabel(feature)
    plt.ylabel("Frequency")
    plt.show()

# 10. Tính hệ số tương quan giữa các biến
correlations = df_numeric.corr()
print("\n📌 Hệ số tương quan giữa y và các feature:")
print(correlations.iloc[-1, :-1])  # In hệ số tương quan của y với các feature

# 11. Chuẩn hóa dữ liệu
# a. Minmax scaling (đưa dữ liệu về khoảng [0,1])
X_minmax = (df_numeric.iloc[:, :-1] - df_numeric.iloc[:, :-1].min()) / (df_numeric.iloc[:, :-1].max() - df_numeric.iloc[:, :-1].min())

# b. Standardization (chuẩn hóa sao cho mean=0, variance=1)
X_standardized = (df_numeric.iloc[:, :-1] - df_numeric.iloc[:, :-1].mean()) / df_numeric.iloc[:, :-1].std()

# 12. Chuyển đổi dữ liệu thành X1 (min=1, max=10)
X1 = df_numeric.iloc[:, :-1].apply(lambda x: 1 + 9 * (x - x.min()) / (x.max() - x.min()))

# 13. Chuyển đổi dữ liệu thành X2 bằng chuẩn hóa (standardize)
X2 = X_standardized.copy()

# 14. In kết quả để kiểm tra
print("\n📌 Dữ liệu sau MinMax Scaling:\n", X_minmax.head())
print("\n📌 Dữ liệu sau Standardization:\n", X_standardized.head())
print("\n📌 Dữ liệu sau chuyển đổi thành X1:\n", X1.head())
print("\n📌 Dữ liệu sau chuẩn hóa thành X2:\n", X2.head())


In [ ]:
# TASK 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1️⃣ Đọc dữ liệu từ file (giữ lại các cột cần thiết)
df = pd.read_csv("50_Startups.csv")
df_numeric = df.select_dtypes(include=[np.number])  # Chỉ lấy cột số

# 2️⃣ Xác định X1 (các feature) và y1 (profit)
X1 = df_numeric[['R&D Spend', 'Administration', 'Marketing Spend']].values  # Chuyển thành mảng numpy
y1 = df_numeric['Profit'].values.reshape(-1, 1)  # Chuyển thành vector cột

# 3️⃣ Chuẩn hóa dữ liệu X1 (Standardization)
X1 = (X1 - np.mean(X1, axis=0)) / np.std(X1, axis=0)

# 4️⃣ Thêm cột bias (1) vào X1 để tính toán b dễ dàng hơn
X1 = np.c_[X1, np.ones(X1.shape[0])]

# 5️⃣ Hàm tính Loss Function L(a, b) (Mean Squared Error)
def compute_loss(y_hat, y):
    m = len(y)
    return np.sum((y_hat - y) ** 2) / (2 * m)

# 6️⃣ Gradient Descent để tìm a1, a2, a3, b
def gradient_descent(X, y, alpha=0.01, iterations=1000):
    m, n = X.shape
    theta = np.zeros((n, 1))  # Khởi tạo hệ số a1, a2, a3, b
    loss_history = []

    for i in range(iterations):
        y_hat = X @ theta  # Tính giá trị dự đoán
        gradient = (X.T @ (y_hat - y)) / m  # Gradient của Loss function
        theta -= alpha * gradient  # Cập nhật hệ số
        loss_history.append(compute_loss(y_hat, y))  # Lưu loss vào history

    return theta, loss_history

# 7️⃣ Chạy gradient descent để tìm a1, a2, a3, b
theta_optimal, loss_history = gradient_descent(X1, y1, alpha=0.01, iterations=500)

# 8️⃣ Vẽ đồ thị loss function theo số bước lặp
plt.plot(range(len(loss_history)), loss_history, color='red')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Loss function over iterations')
plt.show()

# 9️⃣ Tính giá trị dự đoán y_hat
y_hat = X1 @ theta_optimal

# 🔟 Vẽ scatter plot giữa y_hat và y thực tế
plt.scatter(y1, y_hat, alpha=0.7)
plt.xlabel("Actual Profit (y)")
plt.ylabel("Predicted Profit (y_hat)")
plt.title("Predicted Profit vs Actual Profit")
plt.show()

# 1️⃣1️⃣ Đánh giá độ chính xác của mô hình bằng RMSE, MAE, R²
rmse = np.sqrt(mean_squared_error(y1, y_hat))
mae = mean_absolute_error(y1, y_hat)
r2 = r2_score(y1, y_hat)

print(f"📌 RMSE: {rmse:.2f}")
print(f"📌 MAE: {mae:.2f}")
print(f"📌 R² Score: {r2:.4f}")
